### Matrix Factorization with Alternating Least Squares

### Springboard Capstone 2 project: building a recommendation engine
### John Burt


### Purpose of this notebook:

Generate an ALS filled ratings matrix of user (rows) x games (cols)


#### The method:

- Load data into a pandas dataframe from provided csv files.

- Use pivot to convert the data into a games(rows) X users(cols) rating matrix, with NaNs where users haven't rated games (majority of cells).

- Drop users who rated too few games, or gave outlier ratings.

- Fit ALS to fill in ratings for all users

- Save ALS filled array



## Load input data



In [3]:
# remove warnings
import warnings
warnings.filterwarnings('ignore')
# ---

%matplotlib inline
import pandas as pd
pd.options.display.max_columns = 100
from matplotlib import pyplot as plt
import matplotlib
matplotlib.style.use('ggplot')
import numpy as np

from datetime import datetime

pd.options.display.max_rows = 100

srcdir = './data/'

# load the boardgame title data
titledata = pd.read_csv(srcdir+'bgg_gamelist.csv')

# rename the gameID column
titledata=titledata.rename(columns = {"id":'gameID'})
titledata.reset_index(inplace=True)

utility_df = pd.read_hdf(srcdir+'bgg_game_mx_unfilled_v2.h5', 'mx')

# should not be necessary: gameID should already be index
# rp_filt = rp_filt.set_index('gameID')

print(titledata.shape)
print(utility_df.shape)
utility_df.head()

(12600, 7)
(12120, 65168)


,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,...,65119,65120,65121,65122,65123,65124,65125,65126,65127,65128,65129,65130,65131,65132,65133,65134,65135,65136,65137,65138,65139,65140,65141,65142,65143,65144,65145,65146,65147,65148,65149,65150,65151,65152,65153,65154,65155,65156,65157,65158,65159,65160,65161,65162,65163,65164,65165,65166,65167,65168
gameID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,NaN,5.0,NaN,NaN,NaN,9.0,NaN,NaN,NaN,NaN,NaN,7.7,NaN,9.0,NaN,NaN,NaN,NaN,10.0,NaN,NaN,8.0,NaN,NaN,NaN,NaN,NaN,NaN,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.0,6.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.0,NaN,NaN,8.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,4.0,NaN,8.0,9.0,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.0,NaN,7.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.0,NaN,NaN,7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.0,NaN,NaN,NaN,8.0,8.0,7.0,NaN,NaN,NaN,7.0,NaN,NaN,...,NaN,NaN,NaN,8.0,9.0,6.5,8.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.0,8.0,NaN,NaN,NaN,NaN,NaN,8.0,8.50,NaN,NaN,NaN,7.5,NaN,9.0,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.0,NaN,NaN,NaN,9.5,8.0,NaN,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.0,NaN,NaN,NaN,NaN,8.0,NaN,NaN,NaN,NaN,8.0,NaN,NaN,...,NaN,NaN,NaN,7.0,NaN,NaN,7.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.0,NaN,NaN,NaN,8.0,NaN,NaN,NaN,6.0,8.0,NaN,NaN,NaN,NaN,NaN,NaN,9.0,NaN,NaN,NaN,NaN,NaN,7.8,8.69,NaN,NaN,7.0,7.3,NaN,9.0,NaN,NaN,6.0,7.0,NaN,NaN


## Alternating Least Squares matrix factorization

### Use ALS to compute the missing ratings.


[Here is a useful description of this method.](https://bugra.github.io/work/notes/2014-04-19/alternating-least-squares-method-for-collaborative-filtering/)

Notes: 

- The ALS algorithm I'm using is from an OSS package called implicit. This is wrapped in a utility function in recsys_utilities.py: do_ALS_df(), which takes a dataframe as input and outputs an ALS filled ratings dataframe with same indices and column names.


- The parameters used for the ALS were determined using hyperparameter tuning in a separate notebook.

In [4]:
# import utility functions
import sys
sys.path.append('./')
import recsys_utilities
import importlib
importlib.reload(recsys_utilities)
from recsys_utilities import do_ALS_df

# model defaults (from HP tuning runs)
params = {
    'regularization': 0.3,
    'n_factors': 25,
    'n_iterations': 20,
    'weighted': False,
    'bm25_K1': 4,
    'bm25_B': 0.5,
    'verbose': True,
    'scale': True,
    'use_native': True,
    'use_cg': True,
    'use_gpu': False,
    }

utility_df_filled, item_factors, user_factors = do_ALS_df(
    utility_df, ALS_method='implicit', **params)


fitting ALS model


## Save the ALS filled matrix to an HDF5 file

In [5]:
# user_ratings_filled.to_csv(srcdir+'bgg_game_mx_filled.csv')
utility_df_filled.to_hdf(srcdir+'bgg_game_mx_filled_v2.h5', key='mx')


In [6]:
utility_df_filled.shape

(12120, 65168)

In [7]:
utility_df_filled.tail()

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,...,65119,65120,65121,65122,65123,65124,65125,65126,65127,65128,65129,65130,65131,65132,65133,65134,65135,65136,65137,65138,65139,65140,65141,65142,65143,65144,65145,65146,65147,65148,65149,65150,65151,65152,65153,65154,65155,65156,65157,65158,65159,65160,65161,65162,65163,65164,65165,65166,65167,65168
gameID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
276925,6.386015,6.207924,6.414303,6.157567,6.258170,6.351043,6.160162,6.075673,5.949022,6.152587,6.151018,6.211317,6.037933,6.462116,6.277407,5.959018,6.484363,5.871089,6.081823,6.055026,6.198666,6.548743,6.227896,5.919703,5.996846,6.212459,6.242943,6.240223,5.854299,6.328776,6.422137,5.923799,6.332831,6.220666,6.239967,6.222564,6.220427,6.138691,5.794073,6.249036,6.137474,5.937656,5.970498,6.147401,6.265683,6.006303,6.165143,6.542853,6.131458,6.464466,...,6.270455,6.049587,6.141913,6.441819,6.426351,6.126408,7.062617,6.116514,6.178725,6.174303,6.293387,6.375491,5.996103,6.130337,6.312321,5.961359,5.969360,6.779797,6.216163,6.608896,5.955475,6.425298,6.240215,6.229632,6.491464,6.031808,6.044845,6.235034,6.254427,6.030169,6.172724,6.247353,5.995852,5.972504,6.692282,5.874828,7.078767,6.288208,6.147497,6.576926,6.079972,6.047498,6.392118,6.143959,6.162734,6.340460,6.283920,6.056695,6.194838,6.186833
277721,6.538056,5.872683,6.256649,6.154757,6.401428,6.255696,6.286851,6.161515,6.055374,6.107759,6.155159,6.202503,6.044830,6.316350,6.476088,6.302219,6.560421,6.059654,6.191854,6.016574,6.182189,6.388591,6.045140,5.982573,6.142472,6.238828,5.930999,6.181767,5.578951,6.224669,6.489919,5.996889,6.044029,6.271877,7.180823,6.139794,6.271113,6.157863,6.041736,6.281436,6.001177,5.704138,6.071423,6.125511,6.077414,6.009338,6.190165,6.183463,5.867213,6.654366,...,6.268713,6.216753,6.375752,6.163857,6.066635,6.150838,7.401213,6.075300,6.207426,6.216413,6.398992,6.320480,6.040918,6.294312,6.376506,6.423843,6.449741,6.293853,6.387267,6.769162,6.321056,6.220976,6.172150,6.202209,6.272469,6.006090,6.109859,6.400149,6.305483,5.805383,6.025389,6.312610,5.922369,6.428540,6.755386,6.265232,7.217903,6.202028,6.341650,6.187694,6.080297,6.040864,6.387999,6.168890,6.067099,6.311953,5.847570,6.186909,6.293118,5.982375
278751,6.105055,5.944846,5.913023,6.134682,6.057880,6.201626,6.319663,6.031420,6.180435,6.195074,6.108746,6.075449,6.250846,6.547694,6.276275,5.825398,6.554298,5.998857,6.401646,6.118531,6.113026,6.494920,6.891368,6.261802,6.157175,6.111492,6.293982,6.083853,6.147938,6.131085,5.940538,6.380602,6.194409,5.849782,6.208471,6.220195,6.132610,6.360270,6.267388,6.383636,6.268568,6.551623,6.232018,6.269504,6.408775,6.211515,5.875174,6.439242,6.852717,6.340019,...,6.152043,6.185471,6.194512,6.402278,6.203137,6.169764,7.527420,6.288509,6.285461,6.171806,6.187458,6.029394,6.103232,6.102353,5.937624,6.146598,5.679831,6.445822,6.231091,5.871800,6.581343,6.050949,6.137367,6.349404,6.049704,6.138336,6.228660,6.519111,6.739342,6.526852,5.826472,6.579005,6.060813,6.023544,6.128255,6.108235,6.524807,6.421551,6.047328,6.149913,5.980931,6.163893,6.206664,6.231037,6.322564,6.081064,6.478848,6.078439,6.352653,6.404754
280794,6.250512,6.011319,6.224489,6.156438,6.285044,6.245353,6.263029,6.142721,6.086185,6.156697,6.203772,6.183410,6.127605,6.201605,6.269678,6.184437,6.435411,5.988534,6.196775,6.138637,6.220085,6.332741,6.168015,6.163512,6.174522,6.264639,6.057923,6.167525,6.028551,6.263186,6.335258,6.033931,6.189750,6.169065,6.271966,6.113391,6.194785,6.068131,6.080978,6.118313,6.005008,6.109583,6.142079,6.175528,6.173938,6.175476,6.169623,6.230261,6.172174,6.188756,...,6.195108,6.127889,6.238698,6.315942,6.220745,6.208078,6.592873,6.096695,6.132159,6.173040,6.274726,6.127547,6.100805,6.105029,6.181401,6.112187,6.096460,6.236363,6.256319,6.208776,6.001191,6.076492,6.152996,6.240991,6.250348,6.239757,6.144095,6.307824,